In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-09-12'
edate = '2022-10-12'
print(f"The analysis is carried our for date range {sdate} to {edate}")

### WAREHOUSE PROFILING AND ANALYSIS

In [ ]:
df=qlib.wh_average_queued_load(start_date="2022-09-29",end_date="2022-09-30",wh_name="DEV_WH")
fig = px.bar(df, x=df["start_time_truncated"], y=df["avg_queued_load"], title=" DEV_WH",color_discrete_sequence=color_scheme)
# Range the width has to be updated in
fig.update_traces(width=2000000)
fig.show()

In [ ]:
df=qlib.queries_by_wh("2022-09-26 14:30:00","2022-09-26 14:50:00")


In [ ]:
df